In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),...,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,2003,3.609987,0.635160,5.999222e+08,2.428732e+09,1.863367e+10,1.124993e+10,3.775011e+10,1.809535e+09,6.162382e+08,...,8.765721,374.017751,0.728082,0.587982,0.574004,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,316.142000
1,2004,3.683781,0.640093,5.851223e+08,2.506090e+09,1.964942e+10,1.175278e+10,4.026419e+10,2.015670e+09,5.396413e+08,...,8.699419,408.908534,0.733483,0.592441,0.580163,2.628414e+07,1.320360e+06,7.602982,2.371561e+07,304.535000
2,2005,3.659977,0.644568,5.590308e+08,2.649361e+09,2.158067e+10,1.256500e+10,4.498692e+10,2.402642e+09,3.968813e+08,...,8.666401,443.392931,0.738287,0.596623,0.586021,2.663983e+07,1.336402e+06,7.628875,2.382047e+07,306.568000
3,2006,3.686649,0.648597,5.264270e+08,2.833246e+09,2.405920e+10,1.373622e+10,5.105978e+10,2.889954e+09,2.186862e+08,...,8.587977,486.763373,0.742506,0.600546,0.591575,2.697126e+07,1.347634e+06,7.653347,2.391566e+07,318.104000
4,2007,3.960873,0.652189,4.920899e+08,3.032448e+09,2.671679e+10,1.531605e+10,5.762426e+10,3.397108e+09,4.320482e+07,...,8.576293,488.841998,0.746151,0.604229,0.596825,2.726833e+07,1.360021e+06,7.672501,2.398904e+07,363.874000
5,2008,4.245857,0.655354,4.607989e+08,3.221668e+09,2.918522e+10,1.735413e+10,6.382184e+10,3.843605e+09,0.000000e+00,...,8.543035,494.115531,0.749235,0.607691,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,446.335000
6,2009,4.359552,0.658104,4.373329e+08,3.375608e+09,3.109628e+10,1.990007e+10,6.879400e+10,4.148947e+09,0.000000e+00,...,8.571316,565.545675,0.751769,0.610950,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,471.072000
7,2010,4.447406,0.660450,4.264713e+08,3.468971e+09,3.208175e+10,2.300349e+10,7.168223e+10,4.232635e+09,0.000000e+00,...,8.539868,637.418103,0.753767,0.614023,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,583.772000
8,2011,4.569912,0.662401,4.329931e+08,3.476458e+09,3.177342e+10,2.671402e+10,7.162801e+10,4.014172e+09,0.000000e+00,...,8.526852,683.859991,0.755240,0.616930,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,742.881000
9,2012,4.858512,0.663873,4.616776e+08,3.372772e+09,2.980306e+10,3.108128e+10,6.777282e+10,3.413059e+09,1.276614e+08,...,12.945391,728.643375,0.756056,0.619689,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,844.864000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),...,INCC (%),Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,3.609987,0.635160,5.999222e+08,2.428732e+09,1.863367e+10,1.124993e+10,3.775011e+10,1.809535e+09,6.162382e+08,5.122550e+09,...,0.895303,8.765721,374.017751,0.728082,0.587982,0.574004,2.590528e+07,1.303625e+06,7.576443,2.359888e+07
1,3.683781,0.640093,5.851223e+08,2.506090e+09,1.964942e+10,1.175278e+10,4.026419e+10,2.015670e+09,5.396413e+08,5.449927e+09,...,0.746570,8.699419,408.908534,0.733483,0.592441,0.580163,2.628414e+07,1.320360e+06,7.602982,2.371561e+07
2,3.659977,0.644568,5.590308e+08,2.649361e+09,2.158067e+10,1.256500e+10,4.498692e+10,2.402642e+09,3.968813e+08,6.065174e+09,...,0.589811,8.666401,443.392931,0.738287,0.596623,0.586021,2.663983e+07,1.336402e+06,7.628875,2.382047e+07
3,3.686649,0.648597,5.264270e+08,2.833246e+09,2.405920e+10,1.373622e+10,5.105978e+10,2.889954e+09,2.186862e+08,6.842013e+09,...,0.606044,8.587977,486.763373,0.742506,0.600546,0.591575,2.697126e+07,1.347634e+06,7.653347,2.391566e+07
4,3.960873,0.652189,4.920899e+08,3.032448e+09,2.671679e+10,1.531605e+10,5.762426e+10,3.397108e+09,4.320482e+07,7.654167e+09,...,0.633401,8.576293,488.841998,0.746151,0.604229,0.596825,2.726833e+07,1.360021e+06,7.672501,2.398904e+07
5,4.245857,0.655354,4.607989e+08,3.221668e+09,2.918522e+10,1.735413e+10,6.382184e+10,3.843605e+09,0.000000e+00,8.375355e+09,...,0.587619,8.543035,494.115531,0.749235,0.607691,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07
6,4.359552,0.658104,4.373329e+08,3.375608e+09,3.109628e+10,1.990007e+10,6.879400e+10,4.148947e+09,0.000000e+00,8.879299e+09,...,0.566759,8.571316,565.545675,0.751769,0.610950,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07
7,4.447406,0.660450,4.264713e+08,3.468971e+09,3.208175e+10,2.300349e+10,7.168223e+10,4.232635e+09,0.000000e+00,9.039721e+09,...,0.593537,8.539868,637.418103,0.753767,0.614023,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07
8,4.569912,0.662401,4.329931e+08,3.476458e+09,3.177342e+10,2.671402e+10,7.162801e+10,4.014172e+09,0.000000e+00,8.730341e+09,...,0.600821,8.526852,683.859991,0.755240,0.616930,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07
9,4.858512,0.663873,4.616776e+08,3.372772e+09,2.980306e+10,3.108128e+10,6.777282e+10,3.413059e+09,1.276614e+08,7.824880e+09,...,0.604731,12.945391,728.643375,0.756056,0.619689,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     304.535000
1     306.568000
2     318.104000
3     363.874000
4     446.335000
5     471.072000
6     583.772000
7     742.881000
8     844.864000
9     844.864000
10    791.375000
11    818.645143
12    676.300114
13    538.586743
14    362.791000
15    340.278000
16    395.359000
17    581.479000
18    576.578000
19    547.622000
20           NaN
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),...,INCC (%),Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-1.362963,-2.001563,1.862218,-1.377227,-0.455887,-1.283032,-1.143593,-0.539011,-0.196467,-0.267673,...,2.660855,-0.870735,-0.922857,-2.181558,-1.902196,-1.804968,-1.598161,-1.331708,-1.837221,-1.785134
1,-1.269604,-1.600297,1.610564,-1.169882,-0.330414,-1.244132,-0.955546,-0.374598,-0.271137,-0.142319,...,1.389772,-0.895279,-0.697647,-1.685140,-1.584630,-1.512744,-1.395552,-0.662133,-1.537870,-1.483445
2,-1.299720,-1.236268,1.166911,-0.785870,-0.091853,-1.181300,-0.602298,-0.065950,-0.410307,0.093262,...,0.050101,-0.907502,-0.475060,-1.243638,-1.286778,-1.234789,-1.205336,-0.020223,-1.245813,-1.212430
3,-1.265976,-0.908613,0.612524,-0.292998,0.214312,-1.090696,-0.148064,0.322729,-0.584021,0.390718,...,0.188826,-0.936534,-0.195117,-0.855941,-1.007331,-0.971219,-1.028091,0.429187,-0.969771,-0.966420
4,-0.919048,-0.616472,0.028666,0.240927,0.542596,-0.968483,0.342942,0.727234,-0.755089,0.701695,...,0.422624,-0.940859,-0.181700,-0.520937,-0.744978,-0.722150,-0.869225,0.924829,-0.753721,-0.776774
5,-0.558509,-0.358983,-0.503399,0.748099,0.847513,-0.810821,0.806504,1.083359,-0.797208,0.977842,...,0.031364,-0.953171,-0.147661,-0.237515,-0.498412,-0.487695,-0.738254,1.322566,-0.619654,-0.691417
6,-0.414670,-0.135285,-0.902408,1.160710,1.083581,-0.613871,1.178409,1.326899,-0.797208,1.170804,...,-0.146903,-0.942701,0.313400,-0.004563,-0.266321,-0.267973,-0.634133,1.422691,-0.559927,-0.649385
7,-0.303524,0.055484,-1.087096,1.410953,1.205313,-0.373795,1.394441,1.393649,-0.797208,1.232231,...,0.081938,-0.954343,0.777316,0.179031,-0.047397,-0.063097,-0.249699,1.321114,-0.123598,-0.183865
8,-0.148538,0.214185,-0.976201,1.431021,1.167226,-0.086755,1.390385,1.219403,-0.797208,1.113767,...,0.144188,-0.959162,1.077084,0.314378,0.159670,0.126817,0.116489,1.046816,0.325768,0.245932
9,0.216576,0.333924,-0.488458,1.153106,0.923834,0.251090,1.102027,0.739957,-0.672757,0.767062,...,0.177601,0.676539,1.366148,0.389375,0.356190,0.310702,0.433902,0.506028,0.511153,0.529790


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     304.535000
1     306.568000
2     318.104000
3     363.874000
4     446.335000
5     471.072000
6     583.772000
7     742.881000
8     844.864000
9     844.864000
10    791.375000
11    818.645143
12    676.300114
13    538.586743
14    362.791000
15    340.278000
16    395.359000
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),...,INCC (%),Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
17,1.264874,1.276504,-0.718272,1.334994,-1.300027,1.151763,1.324067,-0.920678,1.78769,-0.935888,...,0.050004,0.974031,-1.301147,1.03243,1.081079,1.391018,1.124406,-1.136033,1.094431,1.001686


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    581.479
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [26]:
train_input.iloc[:-1]

,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),...,INCC (%),Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-1.362963,-2.001563,1.862218,-1.377227,-0.455887,-1.283032,-1.143593,-0.539011,-0.196467,-0.267673,...,2.660855,-0.870735,-0.922857,-2.181558,-1.902196,-1.804968,-1.598161,-1.331708,-1.837221,-1.785134
1,-1.269604,-1.600297,1.610564,-1.169882,-0.330414,-1.244132,-0.955546,-0.374598,-0.271137,-0.142319,...,1.389772,-0.895279,-0.697647,-1.685140,-1.584630,-1.512744,-1.395552,-0.662133,-1.537870,-1.483445
2,-1.299720,-1.236268,1.166911,-0.785870,-0.091853,-1.181300,-0.602298,-0.065950,-0.410307,0.093262,...,0.050101,-0.907502,-0.475060,-1.243638,-1.286778,-1.234789,-1.205336,-0.020223,-1.245813,-1.212430
3,-1.265976,-0.908613,0.612524,-0.292998,0.214312,-1.090696,-0.148064,0.322729,-0.584021,0.390718,...,0.188826,-0.936534,-0.195117,-0.855941,-1.007331,-0.971219,-1.028091,0.429187,-0.969771,-0.966420
4,-0.919048,-0.616472,0.028666,0.240927,0.542596,-0.968483,0.342942,0.727234,-0.755089,0.701695,...,0.422624,-0.940859,-0.181700,-0.520937,-0.744978,-0.722150,-0.869225,0.924829,-0.753721,-0.776774
5,-0.558509,-0.358983,-0.503399,0.748099,0.847513,-0.810821,0.806504,1.083359,-0.797208,0.977842,...,0.031364,-0.953171,-0.147661,-0.237515,-0.498412,-0.487695,-0.738254,1.322566,-0.619654,-0.691417
6,-0.414670,-0.135285,-0.902408,1.160710,1.083581,-0.613871,1.178409,1.326899,-0.797208,1.170804,...,-0.146903,-0.942701,0.313400,-0.004563,-0.266321,-0.267973,-0.634133,1.422691,-0.559927,-0.649385
7,-0.303524,0.055484,-1.087096,1.410953,1.205313,-0.373795,1.394441,1.393649,-0.797208,1.232231,...,0.081938,-0.954343,0.777316,0.179031,-0.047397,-0.063097,-0.249699,1.321114,-0.123598,-0.183865
8,-0.148538,0.214185,-0.976201,1.431021,1.167226,-0.086755,1.390385,1.219403,-0.797208,1.113767,...,0.144188,-0.959162,1.077084,0.314378,0.159670,0.126817,0.116489,1.046816,0.325768,0.245932
9,0.216576,0.333924,-0.488458,1.153106,0.923834,0.251090,1.102027,0.739957,-0.672757,0.767062,...,0.177601,0.676539,1.366148,0.389375,0.356190,0.310702,0.433902,0.506028,0.511153,0.529790


In [27]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_data=(train_input.iloc[-1:], 
                                         train_target.iloc[-1:]),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [28]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [29]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3588929861, 1019035729, 1351756840, 521575181, 3500419941, 2328873200, 741463557, 133997680, 4069874087, 2127247672, 4161671088, 530975668, 1656585265, 2246257207, 844986232, 3372649990, 3755889088, 2677696337, 2553557574, 1659254824, 3031982806, 3193563697, 2086451060, 1724933591, 518117858, 1403987742, 3616206931, 540988672, 1780886102, 2912193821, 561729448, 4072636571, 756778821, 2049367217, 570140215, 1857111470, 2821058871, 1054657509, 3507535566, 509995370, 3883236246, 448511027, 4237556397, 2747270217, 4287389107, 629067935, 408464064, 342081844, 3548795377, 505220432, 3474700704, 1178875480, 4093084956, 2126181862, 1019540840, 3578686597, 1659734577, 2783995779, 2388231625, 4220884551, 3560499005, 1517392596, 2837542806, 2551281630, 2158635684, 4032764932, 1153573159, 2876539717, 2646111940, 1564132581, 4210036889, 1118545175, 3532219364, 3459690059, 1622618228, 3202443584, 1562573185, 3859061447, 1397224245, 3655417356, 652925390, 3069041154, 2671443793, 2227248736, 31292748

val_loss: 766.27490234375


Step: 83 ___________________________________________
val_loss: 0.0001373291015625


Step: 84 ___________________________________________
val_loss: 899.8114013671875


Step: 85 ___________________________________________
val_loss: 1733.197021484375


Step: 86 ___________________________________________
val_loss: 0.00252324715256691


Step: 87 ___________________________________________
val_loss: 35.56880569458008


Step: 88 ___________________________________________
val_loss: 1063.681884765625


Step: 89 ___________________________________________
val_loss: 0.030396848917007446


Step: 90 ___________________________________________
val_loss: 1.0862946510314941e-05


Step: 91 ___________________________________________
val_loss: 7906.45947265625


Step: 92 ___________________________________________
val_loss: 4646.14404296875


Step: 93 ___________________________________________
val_loss: 252.50808715820312


Step: 94 ________________________________________

In [30]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 468ms/step - loss: 340710.9062 - val_loss: 158296.2344
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 325468.3438 - val_loss: 1817.1951
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 364937.3750 - val_loss: 317333.3438
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 311321.8438 - val_loss: 169068.0781
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 250154.1562 - val_loss: 282459.2188
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 199867.9062 - val_loss: 535036.8125
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 219576.7188 - val_loss: 318652.5312
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 192546.1562 - val_loss: 310115.6250
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 175401.6719 - val_loss: 294751.7812
Epoch 10/10

1/1 [==============================] - 0s 13ms/step - loss: 88083.3672 - val_loss: 156547.8594
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 80212.2031 - val_loss: 162847.3438
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 85111.6953 - val_loss: 152078.2969
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 83905.4375 - val_loss: 114205.4219
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 83504.8750 - val_loss: 137992.2031
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 84443.9453 - val_loss: 142019.6250
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 82880.7344 - val_loss: 140597.8125
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 76791.7031 - val_loss: 134489.7656
Epoch 83/10000
1/1 [==============================] - 0s 18ms/step - loss: 77963.0547 - val_loss: 135842.7656
Epoch 84/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 29536.0527 - val_loss: 34994.3125
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 28775.9375 - val_loss: 34637.2461
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 28368.8652 - val_loss: 34551.0859
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 27930.1797 - val_loss: 34179.3984
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 27464.3359 - val_loss: 33522.0586
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 27068.2656 - val_loss: 32903.0781
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 26683.5430 - val_loss: 32260.3379
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 26308.1406 - val_loss: 31603.1777
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 26011.8945 - val_loss: 31118.0352
Epoch 158/10000
1/1 [=====

Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 13856.5547 - val_loss: 17055.3574
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 16573.2656 - val_loss: 11195.4131
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 15962.5957 - val_loss: 10602.1240
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 15782.4600 - val_loss: 10074.3076
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 15632.8340 - val_loss: 9640.4492
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 15500.7754 - val_loss: 9274.3691
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 15379.2363 - val_loss: 8958.1885
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 15264.3506 - val_loss: 8679.0986
Epoch 232/10000
1/1 [==============================] - 0s 13ms/step - loss: 15153.9902 - val_loss: 8427.9326
Epoch 233/10000

1/1 [==============================] - 0s 13ms/step - loss: 10934.9062 - val_loss: 1505.7777
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 10980.0430 - val_loss: 1501.3970
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 10868.4512 - val_loss: 1413.1111
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 10834.0762 - val_loss: 1365.2740
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 10800.4492 - val_loss: 1319.4799
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 10858.8594 - val_loss: 1326.0039
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 10739.4160 - val_loss: 1242.5023
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 11308.2500 - val_loss: 1369.1626
Epoch 307/10000
1/1 [==============================] - 0s 13ms/step - loss: 10681.8965 - val_loss: 1123.7820
Epoch 308/10000
1/1 [==============

...

1/1 [==============================] - 0s 13ms/step - loss: 13011.0332 - val_loss: 25.3479
Epoch 834/10000
1/1 [==============================] - 0s 13ms/step - loss: 13121.8945 - val_loss: 4.6146
Epoch 835/10000
1/1 [==============================] - 0s 13ms/step - loss: 12917.8203 - val_loss: 46.5500
Epoch 836/10000
1/1 [==============================] - 0s 14ms/step - loss: 12990.0508 - val_loss: 19.9847
Epoch 837/10000
1/1 [==============================] - 0s 15ms/step - loss: 12888.8291 - val_loss: 36.5293
Epoch 838/10000
1/1 [==============================] - 0s 13ms/step - loss: 12928.0029 - val_loss: 30.8657
Epoch 839/10000
1/1 [==============================] - 0s 13ms/step - loss: 12877.9961 - val_loss: 27.3674
Epoch 840/10000
1/1 [==============================] - 0s 13ms/step - loss: 12990.3770 - val_loss: 11.6521
Epoch 841/10000
1/1 [==============================] - 0s 13ms/step - loss: 13133.9551 - val_loss: 1.3603
Epoch 842/10000
1/1 [==============================] - 

Epoch 910/10000
1/1 [==============================] - 0s 13ms/step - loss: 13101.2188 - val_loss: 5.5935
Epoch 911/10000
1/1 [==============================] - 0s 13ms/step - loss: 13248.0273 - val_loss: 152.1746
Epoch 912/10000
1/1 [==============================] - 0s 13ms/step - loss: 13445.0996 - val_loss: 43.2264
Epoch 913/10000
1/1 [==============================] - 0s 13ms/step - loss: 13124.6816 - val_loss: 117.6995
Epoch 914/10000
1/1 [==============================] - 0s 13ms/step - loss: 13078.5469 - val_loss: 93.2768
Epoch 915/10000
1/1 [==============================] - 0s 13ms/step - loss: 13446.2344 - val_loss: 24.7094
Epoch 916/10000
1/1 [==============================] - 0s 13ms/step - loss: 13095.1172 - val_loss: 92.0236
Epoch 917/10000
1/1 [==============================] - 0s 13ms/step - loss: 13451.7148 - val_loss: 22.0632
Epoch 918/10000
1/1 [==============================] - 0s 13ms/step - loss: 13423.2520 - val_loss: 29.2064
Epoch 919/10000
1/1 [===============

1/1 [==============================] - 0s 13ms/step - loss: 13044.0078 - val_loss: 31.0699
Epoch 987/10000
1/1 [==============================] - 0s 13ms/step - loss: 12972.9834 - val_loss: 68.4195
Epoch 988/10000
1/1 [==============================] - 0s 13ms/step - loss: 13035.0518 - val_loss: 38.2633
Epoch 989/10000
1/1 [==============================] - 0s 13ms/step - loss: 13023.5752 - val_loss: 45.7233
Epoch 990/10000
1/1 [==============================] - 0s 13ms/step - loss: 12971.4072 - val_loss: 83.6091
Epoch 991/10000
1/1 [==============================] - 0s 13ms/step - loss: 12895.3887 - val_loss: 116.6964
Epoch 992/10000
1/1 [==============================] - 0s 13ms/step - loss: 13048.0830 - val_loss: 38.7264
Epoch 993/10000
1/1 [==============================] - 0s 13ms/step - loss: 12968.8447 - val_loss: 79.9411
Epoch 994/10000
1/1 [==============================] - 0s 13ms/step - loss: 12960.4922 - val_loss: 76.6897
Epoch 995/10000
1/1 [==============================]

Epoch 1062/10000
1/1 [==============================] - 0s 14ms/step - loss: 12998.4004 - val_loss: 66.0494
Epoch 1063/10000
1/1 [==============================] - 0s 13ms/step - loss: 12906.9551 - val_loss: 125.8905
Epoch 1064/10000
1/1 [==============================] - 0s 13ms/step - loss: 12897.4512 - val_loss: 121.5781
Epoch 1065/10000
1/1 [==============================] - 0s 13ms/step - loss: 12339.8486 - val_loss: 217.5337
Epoch 1066/10000
1/1 [==============================] - 0s 13ms/step - loss: 12906.3262 - val_loss: 123.4358
Epoch 1067/10000
1/1 [==============================] - 0s 13ms/step - loss: 12806.4189 - val_loss: 227.5818
Epoch 1068/10000
1/1 [==============================] - 0s 13ms/step - loss: 12917.2852 - val_loss: 101.4407
Epoch 1069/10000
1/1 [==============================] - 0s 13ms/step - loss: 12893.6855 - val_loss: 109.8300
Epoch 1070/10000
1/1 [==============================] - 0s 13ms/step - loss: 12339.5605 - val_loss: 206.0777
Epoch 1071/10000
1/1

1/1 [==============================] - 0s 13ms/step - loss: 12736.5430 - val_loss: 202.4385
Epoch 1138/10000
1/1 [==============================] - 0s 13ms/step - loss: 12892.2090 - val_loss: 96.0804
Epoch 1139/10000
1/1 [==============================] - 0s 14ms/step - loss: 12753.2188 - val_loss: 204.9473
Epoch 1140/10000
1/1 [==============================] - 0s 13ms/step - loss: 12725.9355 - val_loss: 183.3748
Epoch 1141/10000
1/1 [==============================] - 0s 13ms/step - loss: 12719.0039 - val_loss: 171.7709
Epoch 1142/10000
1/1 [==============================] - 0s 13ms/step - loss: 12888.1934 - val_loss: 83.5918
Epoch 1143/10000
1/1 [==============================] - 0s 13ms/step - loss: 12744.3203 - val_loss: 187.3410
Epoch 1144/10000
1/1 [==============================] - 0s 13ms/step - loss: 12720.7168 - val_loss: 170.0363
Epoch 1145/10000
1/1 [==============================] - 0s 13ms/step - loss: 13115.2539 - val_loss: 241.2008
Epoch 1146/10000
1/1 [================

In [31]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 34ms/step
[581.479](test_target) - [[395.3623]](prediction) = 186.11669531250004


In [32]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [33]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),...,INCC (%),Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-1.068654,-1.700286,1.421053,-1.375614,-1.389276,-1.112474,-1.377968,-1.434925,1.612503,-1.428002,...,2.299346,1.910652,-1.439459,-1.745788,-1.623176,-1.613758,-1.553499,-1.837321,-1.679335,-1.710688
1,-0.841241,-1.107687,1.184805,-1.167277,-1.179957,-0.984162,-1.171115,-1.204377,1.295322,-1.200619,...,0.891536,1.048693,-0.996403,-1.109850,-1.099829,-1.102222,-1.056537,-1.137335,-1.108899,-1.078624
2,-0.914600,-0.570080,0.768312,-0.781429,-0.781981,-0.776910,-0.782541,-0.771574,0.704163,-0.773292,...,-0.592240,0.619444,-0.558507,-0.544263,-0.608971,-0.615665,-0.589975,-0.466270,-0.552362,-0.510827
3,-0.832405,-0.086192,0.247863,-0.286199,-0.271227,-0.478054,-0.282881,-0.226547,-0.033729,-0.233729,...,-0.438592,-0.400118,-0.007773,-0.047602,-0.148443,-0.154288,-0.155226,0.003551,-0.026342,0.004587
4,0.012676,0.345248,-0.300252,0.250280,0.276427,-0.074931,0.257228,0.340671,-0.760384,0.330362,...,-0.179645,-0.552010,0.018622,0.381556,0.283911,0.281706,0.234444,0.521704,0.385357,0.401909
5,0.890911,0.725514,-0.799745,0.759877,0.785102,0.445119,0.767149,0.840048,-0.939292,0.831272,...,-0.612992,-0.984389,0.085588,0.744635,0.690252,0.692116,0.555690,0.937506,0.640832,0.580740
6,1.241287,1.055876,-1.174327,1.174462,1.178917,1.094760,1.176246,1.181552,-0.939292,1.181292,...,-0.810435,-0.616712,0.992634,1.043059,1.072735,1.076739,0.811079,1.042178,0.754646,0.668800
7,1.512026,1.337608,-1.347709,1.425901,1.381995,1.886652,1.413882,1.275152,-0.939292,1.292715,...,-0.556978,-1.025560,1.905297,1.278253,1.433521,1.435372,1.754024,0.935988,1.586103,1.644102


train_target:


,Alagoas - Consumo de Cimento (t)
0,304.535
1,306.568
2,318.104
3,363.874
4,446.335
5,471.072
6,583.772
7,742.881


test_input:


...

,Alagoas - Valor Cimento Portland,Alagoas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),Contratação Jurídico Comercial (Bi R$),Contratação Jurídico FGTS (Bi R$),Contratação Jurídico Livre (Bi R$),...,INCC (%),Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
18,1.797734,1.128154,0.994288,2.394804,-0.631233,1.044637,2.513792,-0.588445,1.362365,-0.32155,...,1.606903,0.865306,-1.150795,0.755725,0.991887,1.293659,0.878738,-0.768557,0.823003,0.751361


test_target:


,Alagoas - Consumo de Cimento (t)
18,576.578


1/1 [==============================] - 0s 35ms/step
Error: 99.62115185546872




[501.431396484375,
 561.6460571289062,
 797.05078125,
 635.7177734375,
 818.6533203125,
 749.8436889648438,
 644.4332885742188,
 578.3248291015625,
 496.76910400390625,
 395.3623046875,
 476.95684814453125]

In [34]:
display(targets)
display(predictions)

[844.864,
 844.864,
 791.375,
 818.6451428571429,
 676.3001142857142,
 538.5867428571428,
 362.791,
 340.278,
 395.359,
 581.479,
 576.578]

[501.431396484375,
 561.6460571289062,
 797.05078125,
 635.7177734375,
 818.6533203125,
 749.8436889648438,
 644.4332885742188,
 578.3248291015625,
 496.76910400390625,
 395.3623046875,
 476.95684814453125]

In [35]:
mae = mean_absolute_error(predictions, targets)
mae

188.7000834580459

In [36]:
porcentage = mae/np.mean(targets)
porcentage

0.3065520797207116